In [1]:
import pandas as pd
import numpy as np

import open3d
# examples/Python/Tutorial/Basic/icp_registration.py
    
from open3d import *
import numpy as np
import copy

In [2]:
path = "voxelling_output/submission_files/check_registration/"
pcw1 = pd.read_csv(path+"0000007_pointcloud.csv", header=None)
pcw2 = pd.read_csv(path+"0000020_pointcloud.csv", header=None)

pcw1.iloc[:, :3].to_csv(path+"source.xyz", sep=" ", header=None, index=None)
pcw2.iloc[:, :3].to_csv(path+"target.xyz", sep=" ", header=None, index=None)

In [3]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    draw_geometries([source_temp, target_temp])

In [4]:
source = read_point_cloud(path+"source.xyz")
target = read_point_cloud(path+"target.xyz")

In [5]:
source

PointCloud with 39281 points.

In [6]:
threshold = 1000000000000000000

trans_init = np.eye(4)
trans_init[:3, 3] = 100*np.random.rand(3)

# trans_init = np.array([[np.cos(np.pi/8), np.sin(np.pi/8), 0., 10.],
#                       [-np.sin(np.pi/8), np.cos(np.pi/8), 0., 20.],
#                       [0,0,1,10],
#                       [0,0,0,1]])
trans_init

array([[ 1.        ,  0.        ,  0.        , 66.46777295],
       [ 0.        ,  1.        ,  0.        , 48.00280859],
       [ 0.        ,  0.        ,  1.        , 63.09584695],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [7]:
draw_registration_result(source, target, trans_init)

In [8]:
print("Initial alignment")
evaluation = evaluate_registration(source, target,
        threshold, trans_init)
print(evaluation)

Initial alignment
RegistrationResult with fitness = 1.000000, inlier_rmse = 65.666740, and correspondence_set size of 39281
Access transformation to get result.


In [9]:
print("Apply point-to-point ICP")
reg_p2p = registration_icp(source, target, threshold, trans_init,
        TransformationEstimationPointToPoint(), )
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
print("")
draw_registration_result(source, target, reg_p2p.transformation)

Apply point-to-point ICP
RegistrationResult with fitness = 1.000000, inlier_rmse = 12.504035, and correspondence_set size of 39281
Access transformation to get result.
Transformation is:
[[ 9.99999688e-01  6.36826445e-04 -4.66460284e-04  2.76984209e+00]
 [-6.36661682e-04  9.99999735e-01  3.53284431e-04  1.99644116e+00]
 [ 4.66685141e-04 -3.52987343e-04  9.99999829e-01 -1.66867856e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



In [12]:
R = reg_p2p.transformation
print(R)
print(R.shape)

[[ 9.99999688e-01  6.36826445e-04 -4.66460284e-04  2.76984209e+00]
 [-6.36661682e-04  9.99999735e-01  3.53284431e-04  1.99644116e+00]
 [ 4.66685141e-04 -3.52987343e-04  9.99999829e-01 -1.66867856e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
(4, 4)


-------------------------------

In [35]:
from utilities.math_utils import extract_rotation, extract_translation

In [36]:
rot = extract_rotation(R)
trans = extract_translation(R)
rot, trans

(array([-0.00035328, -0.00046646, -0.00063683]),
 array([ 2.76984209,  1.99644116, -1.66867856]))

---------------------------------------------

In [37]:
def rot_and_trans_mat_to_euler_3d(R):
    # based on rotation and translation matrix defenition found here: http://planning.cs.uiuc.edu/node104.html
    assert R[3,3]==1 and R.shape==(4,4) and R[3,1]==0
    x_t = R[0,3]
    y_t = R[1,3]
    z_t = R[2,3]
    beta = -np.arcsin(R[2,0])
    gamma = np.arcsin(R[2,1]/np.cos(beta))
    alpha = np.arcsin(R[1,0]/np.cos(beta))
    return (x_t, y_t, z_t), (alpha, beta, gamma)

In [38]:
trans, rot = rot_and_trans_mat_to_euler_3d(R)

In [39]:
rot, trans  ## looks like my function iz good but alpha and gamma should replace positions 

((-0.0006366617942376715, -0.00046668515798245963, -0.0003529873890561033),
 (2.769842092347897, 1.9964411649030667, -1.6686785648441038))

In [44]:
gt_ego1 = pd.read_csv(path+"0000007_egomotion.csv", header=None)
gt_ego2 = pd.read_csv(path+"0000020_egomotion.csv", header=None)
gt_ego1-gt_ego2

FileNotFoundError: File b'E:/Datasets/DataHack/Test/vid_21_estimate_egomotion0000007_egomotion.csv' does not exist

----------------------------------------

In [16]:
from glob import glob
import os
import open3d
from utilities.math_utils import extract_rotation, extract_translation
import numpy as np
import pandas as pd

path = "E:/Datasets/DataHack/Test/vid_21_estimate_egomotion"
trans_init = np.eye(4)
trans_init[:3, 3] = 100*np.random.rand(3)
threshold = 10000

pc_paths = glob(path+'/*pointcloud.csv')
xyz_paths = [i.replace('_pointcloud.csv', '.xyz') for i in pc_paths]

if not os.path.exists(xyz_paths[0]):
    for idx in range(len(pc_paths)):
        pc = pd.read_csv(pc_paths[idx], header=None)
        pc.iloc[:, :3].to_csv(xyz_path[idx], sep=" ", header=None, index=None)

pc_prev = open3d.read_point_cloud(xyz_paths[0])
R_prev = np.eye(4)
cur_ego = pd.DataFrame(np.zeros((1,6)), dtype=np.float)
print(cur_ego)

for xyz_path in xyz_paths[1:]:
    pc_cur = open3d.read_point_cloud(xyz_path)
    evaluation = open3d.evaluate_registration(pc_prev, pc_cur, threshold, trans_init)
    print("Apply point-to-point ICP to: \n{}".format(xyz_path))
    reg_p2p = open3d.registration_icp(pc_prev, pc_cur, threshold, trans_init,
            open3d.TransformationEstimationPointToPoint(), )
    print(reg_p2p)
    print("Transformation is:")
    R_cur = reg_p2p.transformation
    print(R_cur)
    R = np.matmul(R_cur, R_prev)
    rot = extract_rotation(R)
    trans = extract_translation(R)
    cur_ego.at[0, :2] = rot
    cur_ego.at[0, 3:] = trans
    print(cur_ego)
    print("")    
    
    cur_ego.iloc[0, :].T.to_csv(xyz_path.replace('.xyz', '_egomotion.csv'), sep=" ", header=None, index=None)
    R_prev = R
    pc_prev = pc_cur


     0    1    2    3    4    5
0  0.0  0.0  0.0  0.0  0.0  0.0
Apply point-to-point ICP to: 
E:/Datasets/DataHack/Test/vid_21_estimate_egomotion\0004001.xyz
RegistrationResult with fitness = 1.000000, inlier_rmse = 15.988768, and correspondence_set size of 42987
Access transformation to get result.
Transformation is:
[[ 9.99999135e-01 -2.75508964e-04 -1.28624424e-03 -3.03663137e+01]
 [ 2.71744213e-04  9.99995682e-01 -2.92619085e-03 -9.76548338e-01]
 [ 1.28704487e-03  2.92583879e-03  9.99994891e-01 -3.65200365e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
          0         1         2          3         4       5
0  0.002926 -0.001286  0.000276 -30.366314 -0.976548 -0.3652

Apply point-to-point ICP to: 
E:/Datasets/DataHack/Test/vid_21_estimate_egomotion\0004002.xyz
RegistrationResult with fitness = 1.000000, inlier_rmse = 15.258841, and correspondence_set size of 43151
Access transformation to get result.
Transformation is:
[[ 9.99994562e-01 -1.06362065e-0

RegistrationResult with fitness = 1.000000, inlier_rmse = 19.975694, and correspondence_set size of 44140
Access transformation to get result.
Transformation is:
[[ 9.99999173e-01 -4.00345813e-04 -1.22179028e-03 -3.20475562e+01]
 [ 3.96854005e-04  9.99995840e-01 -2.85684863e-03 -3.64603021e-01]
 [ 1.22292892e-03  2.85636139e-03  9.99995173e-01 -5.05044265e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
          0         1         2           3          4          5
0  0.027698 -0.028169  0.002922 -433.615194 -19.632923 -30.094498

Apply point-to-point ICP to: 
E:/Datasets/DataHack/Test/vid_21_estimate_egomotion\0004015.xyz
RegistrationResult with fitness = 1.000000, inlier_rmse = 14.964282, and correspondence_set size of 44274
Access transformation to get result.
Transformation is:
[[ 9.99998974e-01  1.12828908e-04 -1.42837558e-03 -3.19114752e+01]
 [-1.16767713e-04  9.99996191e-01 -2.75775917e-03 -1.16529083e-01]
 [ 1.42805898e-03  2.75792312e-03  9.99995177e

RegistrationResult with fitness = 1.000000, inlier_rmse = 17.329157, and correspondence_set size of 45194
Access transformation to get result.
Transformation is:
[[ 9.99999297e-01  2.22600416e-04 -1.16487635e-03 -3.49623521e+01]
 [-2.26025868e-04  9.99995649e-01 -2.94130932e-03  5.26006184e-03]
 [ 1.16421654e-03  2.94157055e-03  9.99994996e-01 -4.00318905e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
          0         1         2           3          4        5
0  0.054711 -0.039635  0.004286 -826.780664 -27.405824 -52.0312

Apply point-to-point ICP to: 
E:/Datasets/DataHack/Test/vid_21_estimate_egomotion\0004028.xyz
RegistrationResult with fitness = 1.000000, inlier_rmse = 21.324341, and correspondence_set size of 45275
Access transformation to get result.
Transformation is:
[[ 9.99994296e-01 -2.43676761e-04 -3.36870995e-03 -3.32330079e+01]
 [ 2.43665032e-04  9.99999970e-01 -3.89219998e-06  9.71364737e-01]
 [ 3.36871080e-03  3.07134096e-06  9.99994326e-01 

RegistrationResult with fitness = 1.000000, inlier_rmse = 31.551607, and correspondence_set size of 46793
Access transformation to get result.
Transformation is:
[[ 9.99999470e-01 -6.38315377e-04  8.07384223e-04 -3.71650711e+01]
 [ 6.38295697e-04  9.99999796e-01  2.46323107e-05 -5.32084514e-01]
 [-8.07399782e-04 -2.41169478e-05  9.99999674e-01 -4.97183131e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
          0        1         2            3          4          5
0  0.079487 -0.05181  0.005996 -1230.634329 -20.010843 -76.981495

Apply point-to-point ICP to: 
E:/Datasets/DataHack/Test/vid_21_estimate_egomotion\0004041.xyz
RegistrationResult with fitness = 1.000000, inlier_rmse = 23.646126, and correspondence_set size of 46715
Access transformation to get result.
Transformation is:
[[ 9.99999416e-01  4.61422075e-05 -1.07974104e-03 -2.58731005e+01]
 [-4.87057991e-05  9.99997180e-01 -2.37435937e-03  9.09085645e-01]
 [ 1.07962844e-03  2.37441057e-03  9.99996598e

Apply point-to-point ICP to: 
E:/Datasets/DataHack/Test/vid_21_estimate_egomotion\0004053.xyz
RegistrationResult with fitness = 1.000000, inlier_rmse = 31.430956, and correspondence_set size of 49663
Access transformation to get result.
Transformation is:
[[ 9.99947011e-01  1.13840502e-03 -1.02312768e-02 -6.72051106e-01]
 [-1.16857481e-03  9.99994986e-01 -2.94328656e-03  4.46573796e+00]
 [ 1.02278748e-02  2.95508661e-03  9.99943327e-01 -3.43090534e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
          0         1         2            3         4           5
0  0.099094 -0.097949  0.007043 -1586.275014  4.685025 -165.260992

Apply point-to-point ICP to: 
E:/Datasets/DataHack/Test/vid_21_estimate_egomotion\0004054.xyz
RegistrationResult with fitness = 1.000000, inlier_rmse = 25.595498, and correspondence_set size of 50333
Access transformation to get result.
Transformation is:
[[ 9.99988730e-01 -1.11469858e-03 -4.61497331e-03 -2.66289847e+01]
 [ 1.10560017e-03

...

RegistrationResult with fitness = 1.000000, inlier_rmse = 23.259513, and correspondence_set size of 44511
Access transformation to get result.
Transformation is:
[[ 9.99992379e-01 -2.89188407e-03 -2.62286274e-03  1.32825391e+00]
 [ 2.88678568e-03  9.99993941e-01 -1.94553611e-03 -3.67213269e+00]
 [ 2.62847311e-03  1.93794964e-03  9.99994668e-01 -4.25578047e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
         0         1         2             3            4            5
0 -0.07257 -0.956718 -2.208236 -10562.102095  8490.714814 -2505.296548

Apply point-to-point ICP to: 
E:/Datasets/DataHack/Test/vid_21_estimate_egomotion\0004937.xyz
RegistrationResult with fitness = 1.000000, inlier_rmse = 31.545004, and correspondence_set size of 44666
Access transformation to get result.
Transformation is:
[[ 9.99992151e-01 -2.96107461e-03 -2.63237183e-03  4.45162819e+00]
 [ 2.95624288e-03  9.99993942e-01 -1.83750500e-03 -4.88931478e+00]
 [ 2.63779687e-03  1.82970865e-03  9

RegistrationResult with fitness = 1.000000, inlier_rmse = 13.106361, and correspondence_set size of 42422
Access transformation to get result.
Transformation is:
[[ 9.99998637e-01  3.25300330e-05 -1.65076670e-03 -2.71122673e+01]
 [-3.63374429e-05  9.99997339e-01 -2.30647184e-03  3.00231640e+00]
 [ 1.65068728e-03  2.30652869e-03  9.99995978e-01 -1.42961606e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
          0         1         2             3            4            5
0 -0.035041 -0.979784 -2.197237 -10873.888615  8494.341063 -2530.752648

Apply point-to-point ICP to: 
E:/Datasets/DataHack/Test/vid_21_estimate_egomotion\0004950.xyz
RegistrationResult with fitness = 1.000000, inlier_rmse = 13.302170, and correspondence_set size of 41863
Access transformation to get result.
Transformation is:
[[ 9.99999023e-01 -5.49980545e-05 -1.39679495e-03 -2.65829376e+01]
 [ 5.19503254e-05  9.99997618e-01 -2.18188707e-03  2.82860908e+00]
 [ 1.39691163e-03  2.18181237e-03 

RegistrationResult with fitness = 1.000000, inlier_rmse = 12.399425, and correspondence_set size of 41144
Access transformation to get result.
Transformation is:
[[ 9.99999145e-01 -1.19124951e-04 -1.30217727e-03 -3.03654933e+01]
 [ 1.16419039e-04  9.99997834e-01 -2.07786868e-03  1.81368271e+00]
 [ 1.30242197e-03  2.07771531e-03  9.99996993e-01 -7.02372597e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
          0         1         2             3            4            5
0 -0.003106 -1.005757 -2.185394 -11122.540408  8487.183686 -2657.030073

Apply point-to-point ICP to: 
E:/Datasets/DataHack/Test/vid_21_estimate_egomotion\0004963.xyz
RegistrationResult with fitness = 1.000000, inlier_rmse = 15.727880, and correspondence_set size of 41265
Access transformation to get result.
Transformation is:
[[ 9.99999085e-01 -1.62650195e-05 -1.35277609e-03 -2.96370692e+01]
 [ 1.34189319e-05  9.99997787e-01 -2.10386918e-03  2.79503488e+00]
 [ 1.35280731e-03  2.10384910e-03 

RegistrationResult with fitness = 1.000000, inlier_rmse = 25.003208, and correspondence_set size of 39834
Access transformation to get result.
Transformation is:
[[ 9.99971805e-01  1.69955725e-03 -7.31446125e-03 -3.89612230e+01]
 [-1.70086114e-03  9.99998539e-01 -1.72045428e-04  4.50827604e+00]
 [ 7.31415816e-03  1.84481460e-04  9.99973234e-01 -5.00340979e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
          0         1         2             3            4            5
0  0.016179 -1.033806 -2.186478 -11306.428643  8568.265265 -2829.775034

Apply point-to-point ICP to: 
E:/Datasets/DataHack/Test/vid_21_estimate_egomotion\0004976.xyz
RegistrationResult with fitness = 1.000000, inlier_rmse = 13.320336, and correspondence_set size of 39339
Access transformation to get result.
Transformation is:
[[ 9.99950647e-01 -2.16885535e-03 -9.69534564e-03 -1.89867190e+01]
 [ 2.15867362e-03  9.99997108e-01 -1.06050853e-03  4.10027848e-01]
 [ 9.69761769e-03  1.03952710e-03 

RegistrationResult with fitness = 1.000000, inlier_rmse = 13.275679, and correspondence_set size of 38224
Access transformation to get result.
Transformation is:
[[ 9.99990829e-01  5.24212145e-04  4.25045990e-03 -8.15767105e+00]
 [-5.24127371e-04  9.99999862e-01 -2.10585178e-05  2.78457796e+00]
 [-4.25047035e-03  1.88305423e-05  9.99990967e-01  7.81789428e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
          0         1        2             3            4            5
0  0.023773 -1.051723 -2.18451 -11227.428967  8598.096487 -2992.348475

Apply point-to-point ICP to: 
E:/Datasets/DataHack/Test/vid_21_estimate_egomotion\0004989.xyz
RegistrationResult with fitness = 1.000000, inlier_rmse = 15.846591, and correspondence_set size of 37905
Access transformation to get result.
Transformation is:
[[ 9.99996828e-01  2.77261064e-04  2.50324299e-03 -1.39999251e+01]
 [-2.71524669e-04  9.99997337e-01 -2.29163475e-03  2.91316694e+00]
 [-2.50387171e-03  2.29094779e-03  9

-----------------------------

In [ ]:

from functools import partial
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pycpd import rigid_registration
import numpy as np
import time

def visualize(iteration, error, X, Y, ax):
    plt.cla()
    ax.scatter(X[:,0],  X[:,1], X[:,2], color='red', label='Target')
    ax.scatter(Y[:,0],  Y[:,1], Y[:,2], color='blue', label='Source')
    ax.text2D(0.87, 0.92, 'Iteration: {:d}\nError: {:06.4f}'.format(iteration, error), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize='x-large')
    ax.legend(loc='upper left', fontsize='x-large')
    plt.draw()
    plt.pause(0.001)

def main():
    X = np.loadtxt('data/bunny_target.txt')
    Y = np.loadtxt('data/bunny_source.txt') #synthetic data, equaivalent to X + 1

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    callback = partial(visualize, ax=ax)

    reg = rigid_registration(**{ 'X': X, 'Y': Y })
    reg.register(callback)
    plt.show()

if __name__ == '__main__':
    main()